## Hidden Markov Model
##### Feb 4th 2023
##### William Lu

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import csv
from bs4 import BeautifulSoup
from _collections import defaultdict
import json
from sklearn.metrics import accuracy_score

### Task 1: Vocabulary Creation

In [2]:
path  = '/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/train'
df = pd.read_csv(path,on_bad_lines='skip',sep='\t',header= None)
df

,0,1,2
0,1,Pierre,NNP
1,2,Vinken,NNP
2,3,",",","
3,4,61,CD
4,5,years,NNS
...,...,...,...
912090,22,to,TO
912091,23,San,NNP
912092,24,Francisco,NNP
912093,25,instead,RB


In [3]:
# rename the column name 
new_df = df.copy()
new_dff = df.copy()
new_dfff = df.copy()
new_df['Occurance']=new_df.groupby([1])[2].transform(len)

In [4]:
new_df.loc[new_df.Occurance<3,1]="<unk>"
new_df['occurance']=new_df.groupby([1])[2].transform(len)
new_df

,0,1,2,Occurance,occurance
0,1,Pierre,NNP,6,6
1,2,<unk>,NNP,2,32537
2,3,",",",",46476,46476
3,4,61,CD,25,25
4,5,years,NNS,1130,1130
...,...,...,...,...,...
912090,22,to,TO,21305,21305
912091,23,San,NNP,269,269
912092,24,Francisco,NNP,209,209
912093,25,instead,RB,104,104


In [5]:
# rename the column
new_df.rename(columns={1: "word"},inplace=True)
# drop duplicate rows based on column word (making sure just one <unk>)
new_df.drop_duplicates(subset=['word'],inplace=True)
# we store one df that contains the tag for each word
tag_df = new_df.copy()
# drop unessary column, includinh 0, 2, and Occurance
new_df.drop(columns=[0, 2,"Occurance"],inplace=True)
# sort the dataframe by occurance
new_df.sort_values(by=['occurance'],ascending=False,inplace=True)
new_df

,word,occurance
2,",",46476
9,the,39533
17,.,37452
1,<unk>,32537
22,of,22104
...,...,...
211080,Portland,3
211104,W,3
211105,Ayer,3
211957,stamp,3


In [6]:
new_df.reset_index(inplace=True)
new_df.drop(columns=['index'],inplace=True)
# put <unk> to be on the first row of the dataframe
new_df.iloc[3], new_df.iloc[0] = new_df.iloc[0], new_df.iloc[3]
# resort it to make sure <unk> has index 0
new_df.reset_index(inplace=True)
# make sure the first few words have the correct order
new_df.iloc[3], new_df.iloc[1] = new_df.iloc[1], new_df.iloc[3]
new_df.reset_index(inplace=True)
new_df

,level_0,index,word,occurance
0,0,0,<unk>,32537
1,1,3,",",46476
2,2,2,.,37452
3,3,1,the,39533
4,4,4,of,22104
...,...,...,...,...
16915,16915,16915,Portland,3
16916,16916,16916,W,3
16917,16917,16917,Ayer,3
16918,16918,16918,stamp,3


In [7]:
new_df.drop(columns=["level_0"],inplace=True)
# same but it's for word "."
new_df.iloc[3], new_df.iloc[2] = new_df.iloc[2], new_df.iloc[3]
new_df['index'] = new_df.index

In [8]:
# shift column word to be the first column
first_column = new_df.pop('word')
new_df.insert(0, 'word', first_column)
new_df

,word,index,occurance
0,<unk>,0,32537
1,",",1,46476
2,the,2,39533
3,.,3,37452
4,of,4,22104
...,...,...,...
16915,Portland,16915,3
16916,W,16916,3
16917,Ayer,16917,3
16918,stamp,16918,3


In [9]:
new_df.to_csv("vocab_csv.csv",index=False)

In [10]:
np.savetxt('/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/vocab.txt', new_df, delimiter=r'\t ', header=r'\t '.join(new_df.columns.values), fmt='%s', comments='', encoding=None)
print("success")

success


In [11]:
print("Selected threshold for unk words replace ment is 3.")
print("Total size of my vocabulary is 16920 after counting.")
print("There are 32537 <unk> after replacement.")

Selected threshold for unk words replace ment is 3.
Total size of my vocabulary is 16920 after counting.
There are 32537 <unk> after replacement.


### Task 2: Model Learning Hidden Markov Model

In [12]:
new_dfff['Occurance']=new_dfff.groupby([1])[2].transform(len)
new_dfff.loc[new_dfff.Occurance<3,1]="<unk>"
new_dfff['occurance']=new_dfff.groupby([1])[2].transform(len)

In [13]:
new_dfff

,0,1,2,Occurance,occurance
0,1,Pierre,NNP,6,6
1,2,<unk>,NNP,2,32537
2,3,",",",",46476,46476
3,4,61,CD,25,25
4,5,years,NNS,1130,1130
...,...,...,...,...,...
912090,22,to,TO,21305,21305
912091,23,San,NNP,269,269
912092,24,Francisco,NNP,209,209
912093,25,instead,RB,104,104


In [14]:
# we want to grab the frequent word into learning HMM
new_dfff.drop_duplicates(subset=[1],inplace=True)
frequent = new_dfff.loc[new_dfff["Occurance"]>=3][1]
frequent

0             Pierre
2                  ,
3                 61
4              years
5                old
             ...    
906187    Steinhardt
909566       Granges
909586        Tartan
910915         Sasea
910937       Fiorini
Name: 1, Length: 16919, dtype: object

In [15]:
# define dictionaries
transition = {}
emission = {}
tag_count = {}
tag_tag = {}
tag_word = {}

In [16]:
new_dff['Occurance']=new_dff.groupby([1])[2].transform(len)
new_dff.loc[new_dff.Occurance<3,1]="<unk>"
new_dff['occurance']=new_dff.groupby([1])[2].transform(len)
new_dff[1]

0            Pierre
1             <unk>
2                 ,
3                61
4             years
            ...    
912090           to
912091          San
912092    Francisco
912093      instead
912094            .
Name: 1, Length: 912095, dtype: object

In [17]:
new_dff

,0,1,2,Occurance,occurance
0,1,Pierre,NNP,6,6
1,2,<unk>,NNP,2,32537
2,3,",",",",46476,46476
3,4,61,CD,25,25
4,5,years,NNS,1130,1130
...,...,...,...,...,...
912090,22,to,TO,21305,21305
912091,23,San,NNP,269,269
912092,24,Francisco,NNP,209,209
912093,25,instead,RB,104,104


In [18]:
existed_keys = new_df['word'].tolist()
words = new_dff[1].values.tolist()
indices = df[0].values.tolist()
poss = df[2].values.tolist()
for w in range(len(words)-1):
    if(indices[w]<indices[w+1]):
        # we use ~ here because some tags have , or . as tag
        if '('+str(poss[w+1]) + '~' + str(poss[w])+')' in tag_tag:
            tag_tag['('+str(poss[w+1]) + '~' + str(poss[w])+')'] +=1
        else:
            tag_tag['('+str(poss[w+1]) + '~' + str(poss[w])+')'] =1

In [19]:
for w in range(len(words)-1):
    if(indices[w]<indices[w+1]):
    # check e now
        if '('+str(words[w]) + '~' + str(poss[w])+')'in tag_word:
            tag_word['('+str(words[w]) + '~' + str(poss[w])+')'] +=1
        else:
            tag_word['('+str(words[w]) + '~' + str(poss[w])+')'] =1

In [20]:
# setting up start tag 
words_length = len(words)
for i in range (words_length):
    # if index is 1, it means that word is the start of the sentence
    if indices[i]==1:
        if '('+str(poss[i]) + '~' + '<s>'+')' in tag_tag:
            tag_tag['('+str(poss[i]) + '~' + '<s>'+')'] +=1
        else:
            tag_tag['('+str(poss[i]) + '~' + '<s>'+')'] =1

In [21]:
#tag_tag

In [22]:
# count the number of the <s> tag
tag_count['<s>']=0
for i in indices:
    if (i==1):
        # start of the sentence
        tag_count['<s>']+=1

In [23]:
tag_count

{'<s>': 38218}

In [24]:
# count other tag number to be in the denominator
for i in poss:
    if i not in tag_count:
        tag_count[i] = 1
    else:
        tag_count[i] +=1

In [25]:
#tag_count

Now we find the transition and emission matrices.

In [26]:
for i in tag_tag:
    ix = i.split('~')[1][:-1] # we need to exclude the right bracket ')'
    transition[i] = tag_tag[i]/tag_count[ix]    

In [27]:
for i in tag_word:
    ix = i.split('~')[1][:-1]
    emission[i] = tag_word[i]/tag_count[ix]

In [28]:
#tag_tag

In [29]:
#tag_word

In [30]:
#transition

In [31]:
#emission

In [32]:
emission_transition = [emission, transition]
with open('/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/hmm.json', 'w') as output:
    json.dump(emission_transition, output)

In [33]:
print("The number of parameters for transition is:", len(transition))
print("The number of parameters for emission is:",len(emission))

The number of parameters for transition is: 1392
The number of parameters for emission is: 23350


### Task 3: Greedy Decoding with HMM

In [34]:
dev = pd.read_csv('/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/dev',sep='\t',names=["index","word","tag"])
dev

,index,word,tag
0,1,The,DT
1,2,Arizona,NNP
2,3,Corporations,NNP
3,4,Commission,NNP
4,5,authorized,VBD
...,...,...,...
131763,13,join,VB
131764,14,the,DT
131765,15,winning,VBG
131766,16,bidder,NN


In [35]:
dev_word = dev["word"].values.tolist()
dev_index = dev["index"].values.tolist()
dev_tag = dev["tag"].values.tolist()

In [36]:
tag_list = list(tag_count.keys())
wordss = []
tagss= []
dev_word_true = []
dev_tag_true = []

In [37]:
# separte words in to a list, same for tags
for i in range(len(dev)-1):
    if dev_index[i]< dev_index[i+1]:
        wordss.append(dev_word[i])
        tagss.append(dev_tag[i])
    else:
        wordss.append(dev_word[i])
        dev_word_true.append(wordss)
        wordss =[]
        tagss.append(dev_tag[i])
        dev_tag_true.append(tagss)
        tagss=[]

In [38]:
tag_list

['<s>',
 'NNP',
 ',',
 'CD',
 'NNS',
 'JJ',
 'MD',
 'VB',
 'DT',
 'NN',
 'IN',
 '.',
 'VBZ',
 'VBG',
 'CC',
 'VBD',
 'VBN',
 'RB',
 'TO',
 'PRP',
 'RBR',
 'WDT',
 'VBP',
 'RP',
 'PRP$',
 'JJS',
 'POS',
 '``',
 'EX',
 "''",
 'WP',
 ':',
 'JJR',
 'WRB',
 '$',
 'NNPS',
 'WP$',
 '-LRB-',
 '-RRB-',
 'PDT',
 'RBS',
 'FW',
 'UH',
 'SYM',
 'LS',
 '#']

In [39]:
frequent=list(frequent)

In [40]:
len(frequent)

16919

In [134]:
def greedy(stnce):
    tag=[]
    prb=0
    pos = 'unk'
    # check the start of the sentence is unk or not
    if stnce[0] not in frequent:
        stnce[0]='<unk>'
        
    for i in tag_list:
        # some index may not exist so need to use try statement
        # reference: https://www.w3schools.com/python/python_try_except.asp
        try:
            a = transition['('+i+'~'+'<s>'+')']*emission['('+stnce[0]+'~'+i+')']
            if a > prb:
                prb = a
                pos = i
        except:
            pass
    tag.append(pos)
    #check if the word is in the vocabulary
    for i in range(1,len(stnce)):
        if stnce[i] not in frequent:
            stnce[i] = "<unk>"
        prb2 = 0
        pos2 ='unk'
        for j in tag_list:
            # some index may not exist so need to use try statement
            try:
                b = transition['('+j+'~'+tag[-1]+')'] * emission['('+stnce[i]+'~'+j+')']
                if b >prb2:
                    prb2 = b
                    pos2 = j
            except:
                pass
        tag.append(pos2)
    return tag

In [135]:
#dev_word_true

In [136]:
#dev_tag_true

In [137]:
tag_pred=[]
for i in dev_word_true:
    tag_pred.append(greedy(i))
tag_pred= sum(tag_pred, [])
tag_acc = sum(dev_tag_true, [])

In [138]:
accuracy = accuracy_score(tag_acc, tag_pred)
print("The accuracy score for greedy decoding HMM is:",accuracy)

The accuracy score for greedy decoding HMM is: 0.9267405939992865


In [139]:
test = pd.read_csv('/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/test',sep='\t',names=["index","word"])
test

,index,word
0,1,Influential
1,2,members
2,3,of
3,4,the
4,5,House
...,...,...
129649,26,them
129650,27,here
129651,28,with
129652,29,us


In [140]:
word_test_data = []
test_words=test["word"].values.tolist()
test_index = test["index"].values.tolist()
test_words_true=[]
for i in range(len(test)-1):
    if test_index[i]< test_index[i+1]:
        word_test_data.append(test_words[i])
    else:
        word_test_data.append(test_words[i])
        test_words_true.append(word_test_data)
        word_test_data =[]

In [141]:
test_words_true.append(test_words[-30:])

In [142]:
len(test_words_true)

5462

In [143]:
test_words[-30:]

['``',
 'We',
 'want',
 'to',
 'see',
 'Nelson',
 'Mandela',
 'and',
 'all',
 'our',
 'comrades',
 'out',
 'of',
 'prison',
 ',',
 'and',
 'if',
 'we',
 'are',
 "n't",
 'disciplined',
 'we',
 'may',
 'not',
 'see',
 'them',
 'here',
 'with',
 'us',
 '.']

In [144]:
tag_pred_test =[]
for i in test_words_true:
    tag_pred_test.append(greedy(i))
#tag_pred_test

In [145]:
#test_words_true

In [146]:
# make the word and tags into list of tuples
wl = []
for t in test_words_true:
    q= 1
    for i in t:
        wl.append((i,q))
        q+=1
tl =[]
for t in tag_pred_test:
    q= 1
    for i in t:
        tl.append((i,q))
        q+=1

In [147]:
wll= pd.DataFrame(wl,columns = ["word","index"])
tll = pd.DataFrame(tl,columns = ["tag","index2"])

In [148]:
len(test_words_true)

5462

In [149]:
greedy_df = pd.concat([wll,tll],axis=1)
greedy_df = greedy_df.drop('index2', axis=1)
first_column = greedy_df.pop('index')
greedy_df.insert(0, 'index', first_column)
greedy_df

,index,word,tag
0,1,<unk>,NNP
1,2,members,NNS
2,3,of,IN
3,4,the,DT
4,5,House,NNP
...,...,...,...
129649,26,them,PRP
129650,27,here,RB
129651,28,with,IN
129652,29,us,PRP


In [150]:
greedy_df.to_csv("greedy.csv",index=False)
np.savetxt('/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/greedy.out', greedy_df, delimiter='\t',fmt='%s', comments='', encoding=None)
print("success")

success


In [151]:
# add this part for the resubmission
fnameg = "/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/greedy.out"
with open(fnameg, 'r') as fg:
    datag = [line for line in fg]
with open(fnameg, 'w') as fileg:
    for line in datag:
        # make sure we add a line for each sentence. 
        if line[0]=="1" and line[1]=="\t":
            fileg.write("\n")
            fileg.write(line)
        else:
            fileg.write(line)
with open(fnameg, 'r') as f2g:
    data22g = [line for line in f2g]
# make sure the first line is not empty
with open(fnameg, 'w') as foutg:
    foutg.writelines(data22g[1:])
print("correct now!")

correct now!


# Please execute both TWO cells above to get the correct format.

### Task 4: Viterbi Decoding with HMM

In [105]:
def viterbi(stnce):
    if stnce[0] not in frequent:
        stnce[0] = '<unk>'
    # create dictionaries to store cumulative prbabilities of a position of a tag
    TAG = {}
    # create a dictionary to know which tag leads to the max cumulative probability
    last_TAG = {}
    for i in range(len(stnce)):
        TAG[i] = {}
        last_TAG[i] = {}
        
    # for the first position cumulative probability 
    for t in tag_list:
        if '('+t + '~' + '<s>'+')' in transition:
            try:
                TAG[0][t] = transition['('+t + '~' + '<s>'+')'] * emission['('+stnce[0] + '~' + t+')']
            except:
                TAG[0][t] = 0
                
    # make sure we have a start tag at the beginning
    # reference: https://www.w3schools.com/python/python_dictionaries_access.asp and help from peers
    keysss = TAG[0].keys()
    for t in keysss:
        last_TAG[0][t] = '<s>'
    
    # continue find cumulartive probabilities
    for i in range(1, len(stnce)):
        if stnce[i] not in frequent:
            stnce[i] = '<unk>'
            
    for i in range(1, len(stnce)):
        a = TAG[i-1].keys()
        for t in a:
            for wt in tag_list:
                if '('+wt + '~' + t+')' in transition:
                    if wt in TAG[i]:
                        try:
                            # some index may not exist so need to use try statement
                            b = transition['('+wt + '~' + t+')']* TAG[i-1][t] * emission['('+stnce[i] + '~' + wt+')']
                            if  b > TAG[i][wt]:
                                TAG[i][wt] = b
                                last_TAG[i][wt] = t
                        except:
                            pass
                    else:
                        # some index may not exist so need to use try statement
                        try:
                            TAG[i][wt] = transition['('+wt + '~' + t+')']* TAG[i-1][t] * emission['('+stnce[i] + '~' + wt+')']
                            last_TAG[i][wt] = t
                        except:
                            TAG[i][wt] = 0

    # backward propogation
    TAG_pred = []
    TAG_val_list = list(TAG[len(stnce)-1].values())
    TAG_key_list = list(TAG[len(stnce)-1].keys())
    # the highest probability is at the last word, loop backward for previous words
    max_prob = max(TAG[len(stnce)-1].values()) # find the maximum
    max_index = TAG_val_list.index(max_prob) # the index of the maximum
    max_tag = TAG_key_list[max_index]
    TAG_pred.append(max_tag)
    
    # iterate through pre_pos
    for i in range(len(stnce)-1, 0, -1):
        try:
            max_tag = last_TAG[i][max_tag]
            TAG_pred.append(max_tag)
        except:
            # some index/pairs does not exist
            TAG_pred.append('unk')
        
    # TAG_pred is in reverse
    TAG_pred_good = []
    for i in range(len(TAG_pred)-1,-1,-1):
        TAG_pred_good.append(TAG_pred[i])
    return TAG_pred_good

In [106]:
# use viterbi to predict pos for dev
viterbi_pred_dev = []
for w in dev_word_true:
    viterbi_pred_dev.append(viterbi(w))

In [107]:
len(viterbi_pred_dev)

5526

In [108]:
viterbi_pred_dev= sum(viterbi_pred_dev, [])
#viterbi_pred_dev

In [109]:
accuracy_v = accuracy_score(tag_acc, viterbi_pred_dev)
print("The accuracy score for viterbi decoding HMM is:",accuracy_v)

The accuracy score for viterbi decoding HMM is: 0.9436133312081123


In [110]:
tag_pred_test_viterbi=[]
for i in test_words_true:
    tag_pred_test_viterbi.append(viterbi(i))
#tag_pred_test_viterbi

In [111]:
vtl =[]
for t in tag_pred_test_viterbi:
    q= 1
    for i in t:
        vtl.append((i,q))
        q+=1

In [112]:
vtll = pd.DataFrame(vtl,columns = ["tag","index3"])

In [113]:
viterbi_df = pd.concat([wll,vtll],axis=1)
viterbi_df = viterbi_df.drop('index3', axis=1)
first_column2 = viterbi_df.pop('index')
viterbi_df.insert(0, 'index', first_column2)
viterbi_df

,index,word,tag
0,1,<unk>,JJ
1,2,members,NNS
2,3,of,IN
3,4,the,DT
4,5,House,NNP
...,...,...,...
129649,26,them,PRP
129650,27,here,RB
129651,28,with,IN
129652,29,us,PRP


In [114]:
viterbi_df.to_csv("viterbi_correct_format.csv",index=False)
np.savetxt('/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/viterbi.out', viterbi_df, delimiter='\t', fmt='%s', comments='', encoding=None)
print("success")

success


In [115]:
# add this part for resubmission
fname = "/Users/William/Downloads/natural-language-processing/Hidden_Markov_Model/data/viterbi.out"
with open(fname, 'r') as f:
    data = [line for line in f]
with open(fname, 'w') as file:
    for line in data:
        
        # make sure we add a line for each sentence. 
        if line[0]=="1" and line[1]=="\t":
            file.write("\n")
            file.write(line)
        else:
            file.write(line)
# make sure the first line is not empty
with open(fname, 'r') as f2:
    data22 = [line for line in f2]
with open(fname, 'w') as fout:
    fout.writelines(data22[1:])
print("correct now!")

correct now!


# Please execute both TWO cells above to get the correct format.